In [230]:
from my_utils import stardisting as sd
from my_utils import tile_processing as tp

import os
import pandas as pd
import numpy as np
from stardist.matching import matching_dataset as scorer
from tqdm import tqdm

In [231]:
path_jhu = "Z:\Interpolations\R - 20x - __\JHU"
path_imgs, path_msks = os.path.join(path_jhu, "images"), os.path.join(path_jhu, "masks")
full_dataset = tp.TileSetReader([path_imgs, path_msks], ['.tif', '.tif']).tile_sets
# [0]-basenames, [1][0]-images, [1][1]-gt, need to add [1][2]-pred

In [232]:
path_models_sd = r"Z:\Segmentation Models\StarDist"
inputs_csv = 'Retrain Tissue Specific.csv'
df = pd.read_csv(os.path.join(path_models_sd, inputs_csv))
df

,Model,Initial Model,Tissue,Center Punch,Random State,Epochs,Learning Rate,Aug Intensity,Aug Hue,Aug Blur,Tau,TP,FP,FN,Precision,Recall,F1-Score,Panoptic Quality
0,Model 61,SD_HE_20x,FallopianTube,256,7,0,0.0003,False,False,False,0.5,1376.0,301.0,844.0,0.820513,0.619820,0.706184,0.516724
1,Model 62,SD_HE_20x,Pancreas,256,7,0,0.0003,False,False,False,0.5,1249.0,305.0,428.0,0.803732,0.744782,0.773135,0.596905
2,Model 63,SD_HE_20x,Skin,256,7,0,0.0003,False,False,False,0.5,1111.0,322.0,685.0,0.775297,0.618597,0.688139,0.518253
3,Model 64,SD_HE_20x,FallopianTube,256,7,10,0.0003,False,False,False,0.5,1600.0,390.0,620.0,0.804020,0.720721,0.760095,0.556599
4,Model 65,Model 64,FallopianTube,256,7,10,0.0003,False,False,False,0.5,1538.0,593.0,682.0,0.721727,0.692793,0.706964,0.511160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Model 219,SD_HE_20x,Pancreas,384,7,-1,0.0003,False,False,False,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,Model 220,SD_HE_20x,Pancreas,512,7,-1,0.0003,False,False,False,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,Model 221,SD_HE_20x,Skin,256,7,-1,0.0003,False,False,False,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,Model 222,SD_HE_20x,Skin,384,7,-1,0.0003,False,False,False,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
def center_punch(image, punch_size):
    center_x = image.shape[0] // 2
    center_y = image.shape[1] // 2
    start_x = center_x - punch_size // 2
    end_x = center_x + punch_size // 2
    start_y = center_y - punch_size // 2
    end_y = center_y + punch_size // 2
    return image[start_x:end_x, start_y:end_y]

In [234]:
for i, row in tqdm(df[df.isna().any(axis=1)].iterrows(), total=len(df[df.isna().any(axis=1)])):
    model_name, starting_model, tissue, punch, r_state, epochs, lr, intensity, hue, blur, tau = [None] * 11
    for item, value in row.items():
        if item == 'Model':
            model_name = value
        elif item == 'Initial Model':
            starting_model = value
        elif item == 'Tissue':
            tissue = value
        elif item == 'Center Punch':
            punch = value
        elif item == 'Random State':
            r_state = value
        elif item == 'Epochs':
            epochs = value
        elif item == 'Learning Rate':
            lr = value
        elif item == 'Aug Intensity':
            intensity = value
        elif item == 'Aug Hue':
            hue = value
        elif item == 'Aug Blur':
            blur = value
        elif item == 'Tau':
            tau = value
    
    if starting_model == '2D_versatile_he':
        model = sd.load_published_he_model(path_models_sd, model_name)
    elif starting_model == 'Random Initialization':
        model = sd.load_random_he_model(path_models_sd, model_name)
    elif starting_model == 'SD_HE_20x':
        model = sd.load_model(os.path.join(path_models_sd, 'Model 49'),
                              os.path.join(path_models_sd, model_name), from_last_weights=False)
    else:
        model = sd.load_model(os.path.join(path_models_sd, starting_model),
                              os.path.join(path_models_sd, model_name), from_last_weights=True)
    
    trn_base = {'Images': [], 'Masks': []}
    trn = {'Images': [], 'Masks': []}
    vld = {'Images': [], 'Masks': []}
    tst = {'Images': [], 'Masks': [], 'Predictions': []}
    for j in range(len(full_dataset[0])):
        name, img, msk = full_dataset[0][j], full_dataset[1][0][j], full_dataset[1][1][j]
        if tissue in name:
            if 'train' in name:
                trn_base['Images'].append(center_punch(img, punch))
                trn_base['Masks'].append(center_punch(msk, punch))
            else:
                tst['Images'].append(img)
                tst['Masks'].append(msk)
    
    # 1 original set gets augmented into 8. Val is taken as the 8th.
    np.random.seed(r_state)
    
    for flp in [False, True]:
        for k_rot in [0, 1, 2, 3]:
            for j in range(len(trn_base['Images'])):
                img, msk = trn_base['Images'][j], trn_base['Masks'][j]
                if flp:
                    img = np.flipud(img)
                    msk = np.flipud(msk)
                img = np.rot90(img, k_rot, (0, 1))
                msk = np.rot90(msk, k_rot, (0, 1))
                augmenter = tp.TilePairAugmenter(img, msk, random_state=r_state+j, rotate90=False, flip=False, intensity=intensity, hue=hue, blur=blur)
                if flp and k_rot == 3:  # last one
                    vld['Images'].append(augmenter.image_rgb)
                    vld['Masks'].append(augmenter.mask_gray)   
                else:
                    trn['Images'].append(augmenter.image_rgb)
                    trn['Masks'].append(augmenter.mask_gray)    
    
    if epochs > 0:
        model = sd.configure_model_for_training(model=model, use_gpu=True, epochs=epochs, learning_rate=lr)
        model = sd.normalize_train_and_threshold(model=model, training_images=trn['Images'], training_masks=trn['Masks'],
                                                 validation_images=vld['Images'], validation_masks=vld['Masks'])
    elif epochs < 0:
        model.optimize_thresholds(vld['Images'], vld['Masks'])
    
    for img in tst['Images']:
        img = img / 255
        pred, _ = model.predict_instances(img)
        tst['Predictions'].append(pred)
    
    scores = scorer(tst['Masks'], tst['Predictions'], thresh=tau, show_progress=False)._asdict()
    df.at[i, 'TP'] = int(scores['tp'])
    df.at[i, 'FP'] = int(scores['fp'])
    df.at[i, 'FN'] = int(scores['fn'])
    df.at[i, 'Precision'] = scores['precision']
    df.at[i, 'Recall'] = scores['recall']
    df.at[i, 'F1-Score'] = scores['f1']
    df.at[i, 'Panoptic Quality'] = scores['panoptic_quality']

  0%|          | 0/9 [00:00<?, ?it/s]

Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.694668575772041, nms=0.3) 


base.py (395): Predicting on non-float input... ( forgot to normalize? )

NMS threshold = 0.3:  80%|████████  | 16/20 [00:01<00:00,  8.20it/s, 0.508 -> 0.001]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:03<00:01,  4.51it/s, 0.616 -> 0.001]

NMS threshold = 0.5:  80%|████████  | 16/20 [00:05<00:01,  2.69it/s, 0.503 -> 0.000]


Using optimized values: prob_thresh=0.506578, nms_thresh=0.3.
Saving to 'thresholds.json'.


 11%|█         | 1/9 [00:12<01:42, 12.77s/it]

...

 89%|████████▉ | 8/9 [03:05<00:22, 22.40s/it]

Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.694668575772041, nms=0.3) 



NMS threshold = 0.3:  80%|████████  | 16/20 [00:08<00:02,  1.78it/s, 0.503 -> 0.000]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:11<00:03,  1.28it/s, 0.693 -> 0.000]

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:18<00:06,  1.24s/it, 0.706 -> 0.000]


Using optimized values: prob_thresh=0.708204, nms_thresh=0.5.
Saving to 'thresholds.json'.


100%|██████████| 9/9 [03:45<00:00, 25.09s/it]


In [235]:
df

,Model,Initial Model,Tissue,Center Punch,Random State,Epochs,Learning Rate,Aug Intensity,Aug Hue,Aug Blur,Tau,TP,FP,FN,Precision,Recall,F1-Score,Panoptic Quality
0,Model 61,SD_HE_20x,FallopianTube,256,7,0,0.0003,False,False,False,0.5,1376.0,301.0,844.0,0.820513,0.619820,0.706184,0.516724
1,Model 62,SD_HE_20x,Pancreas,256,7,0,0.0003,False,False,False,0.5,1249.0,305.0,428.0,0.803732,0.744782,0.773135,0.596905
2,Model 63,SD_HE_20x,Skin,256,7,0,0.0003,False,False,False,0.5,1111.0,322.0,685.0,0.775297,0.618597,0.688139,0.518253
3,Model 64,SD_HE_20x,FallopianTube,256,7,10,0.0003,False,False,False,0.5,1600.0,390.0,620.0,0.804020,0.720721,0.760095,0.556599
4,Model 65,Model 64,FallopianTube,256,7,10,0.0003,False,False,False,0.5,1538.0,593.0,682.0,0.721727,0.692793,0.706964,0.511160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,Model 219,SD_HE_20x,Pancreas,384,7,-1,0.0003,False,False,False,0.5,1102.0,161.0,575.0,0.872526,0.657126,0.749660,0.587593
159,Model 220,SD_HE_20x,Pancreas,512,7,-1,0.0003,False,False,False,0.5,1281.0,386.0,396.0,0.768446,0.763864,0.766148,0.589814
160,Model 221,SD_HE_20x,Skin,256,7,-1,0.0003,False,False,False,0.5,1150.0,389.0,646.0,0.747238,0.640312,0.689655,0.517675
161,Model 222,SD_HE_20x,Skin,384,7,-1,0.0003,False,False,False,0.5,1186.0,488.0,610.0,0.708483,0.660356,0.683573,0.511343


In [236]:
df.to_csv(os.path.join(path_models_sd, inputs_csv), index=False)